In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

## What is estimate breadth?

$\textrm{estimate breadth} = \frac{\textrm{estimate upgrades - estimate downgrades}}{\textrm{total number of estimates}}$

## Why is it not sufficient on its own?

Consider the case of the following 3 Amazon review histories for three sellers:
    * 2/2 positive reviews. 100 % positive
    * 9/10 positive reviews. 90 % positive 
    * 88/100 postiive reviews. 88% positive

Clearly not all of the review histories are equal, and you would probably feel that seller number 3 is going to give you the best experience. But how do we incorporate this in mathematically?

With a Bayesian prior obviously... With a uniform prior (i.e. one additional negative and positive review), we can retrieve a postierior that is closer to our real beliefs. Taking the first seller as an example:

$$\textrm{(Prior 1)     Bin} \sim (\hat{p} = \frac{1}{2}, n = 2)$$
$$\textrm{(Prior 2)      Bin} \sim (\hat{p} = \frac{2}{2}, n = 2)$$

$$\textrm{(Posterior)      Beta} \sim (\hat{\alpha} = \frac{3}{4}, \hat{\beta} = \frac{1}{4})$$

$$\hat{\mathbb{E}}[\textrm{Success}_{\textrm{post}}] = \frac{\frac{3}{4} + \frac{1}{4}}{\frac{3}{4}} = 75\%$$

Whereas it is $87.2\%$ for seller 3.

Thats pretty neat. The true optimal solution is actually the center of mass for the beta PDF... but we do not want to do any hectic maths in the calculation, so this quick and dirty alternative will work great for us.

## Laplace breadth

I propose we call it Laplace breadth because although pastor Bayes discovered the formula, he was a layman, and his work gained no notice until Laplace dragged it kicking and screaming into 20th century maths along with the rest of probability theory. And Laplace even proposed a similar thought experiment in his own writings.

The revised estimate breadth formula is:

$$\textrm{laplace breadth} = \frac{(\textrm{estimate upgrades} + 1) - (\textrm{estimate downgrades} + 1)}{\textrm{total estimates} + 2}$$

$$\textrm{laplace breadth} = \frac{(\textrm{estimate upgrades}) - (\textrm{estimate downgrades})}{\textrm{total estimates} + 2}$$